In [1]:
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.graph_objects as go
import numpy as np
from functools import reduce
import time
import pandas as pd
from scipy import stats


In [2]:
df = pd.read_csv('real01.csv')
df.head()

C:\Users\jugal\AppData\Local\Temp\ipykernel_25288\2079683837.py:1: DtypeWarning: Columns (1,2,3,5,6,7,8) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv('real01.csv')


,client_id,clnt_tenure_yr,clnt_tenure_mnth,clnt_age,gendr,num_accts,bal,calls_6_mnth,logons_6_mnth,clnt_tenure_yr_status,...,calls_6_mnth_status,logons_6_mnth_status,Variation,visitor_id,visit_id,process_step,date_time,Test,Control,Missing_variation
0,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Provided,...,Provided,Provided,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:51:13,1,0,0
1,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Provided,...,Provided,Provided,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:47:50,1,0,0
2,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Provided,...,Provided,Provided,Test,427070339_1413275162,228976764_46825473280_96584,confirm,2017-04-02 11:46:45,1,0,0
3,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Provided,...,Provided,Provided,Test,427070339_1413275162,228976764_46825473280_96584,step_3,2017-04-02 11:23:08,1,0,0
4,836976,6.0,73.0,60.5,U,2.0,45105.3,6.0,9.0,Provided,...,Provided,Provided,Test,427070339_1413275162,228976764_46825473280_96584,step_2,2017-04-02 11:22:24,1,0,0


### 1. Completion Rate

In [ ]:
# H0:  No significant difference in completion rates between the Test and Control group
# H1:  Significant difference in completion rates between the Test and Control group

In [3]:
# Completion rate
test_completion_rate = df['Test'].mean()
control_completion_rate = df['Control'].mean()

In [4]:
# Sample sizes
n_test = n_control = len(df)

# Pooled sample proportion
p_pooled = (df['Test'].sum() + df['Control'].sum()) / (n_test + n_control)

# Standard error

se = np.sqrt(p_pooled * (1 - p_pooled) * (2/n_test)) # Modified for paired data

#Z-statistic
z_stat = (test_completion_rate - control_completion_rate) / se

# p-value
p_value = stats.norm.sf(abs(z_stat)) * 2 # two-tailed set

print(f"Test group completion rate: {test_completion_rate:.4f}")
print(f"Control group completion rate: {control_completion_rate:.4f}")
print(f"Z-statistic: {z_stat:.4f}")
print(f"P-value: {p_value:.4f}")

# Decision
alpha = 0.05
if p_value < alpha:
    print("Reject the null hypothesis. There is a significant difference in completion rates.")
else:
    print("Fail to reject the null hypothesis. There is not enough evidence to conclude a significant difference.")



Test group completion rate: 0.3954
Control group completion rate: 0.3189
Z-statistic: 75.6573
P-value: 0.0000
Reject the null hypothesis. There is a significant difference in completion rates.


### Conclusion:

P-value is extremely small(much less than our significance level of 0.05) which provides the evidence to reject the null hypothesis. Therfore, we conclude that there is a statistically significant difference in completion rates between the test and control groups. The new UI design (Test group) demonstrates a significantly different completion rate compared to the old design (Control group)

In [5]:
fig = go.Figure(data = [
    go.Bar(name='Test Group', x=['Completion Rate'], y=[0.3954]),
    go.Bar(name='Control Group', x =['Completion Rate'], y=[0.3189])

])

fig.update_layout(
    title='Completion Rate Comparision: Test vs Control Group',
    yaxis_title = 'Completion Rate',
    barmode = 'group'
)

fig.show()

### 2. Completion Rate with a Cost-Effectiveness Threshold


In [6]:
#H0: Difference in compltetion rates (Test - Control) <= 5%
#H1: Difference in compltetion rates (Test - Control) >= 5%

In [6]:
observed_diff = test_completion_rate - control_completion_rate

# Calculatinog the pooled sample proportion
p = (df['Test'].sum() + df['Control'].sum()) / (n_test + n_control)

# Standard error calculation
n = len(df)
se = np.sqrt(p * (1 - p) * (2/n))# Modified for paired data

#Z-statistic
z = (observed_diff - alpha) / se

# P- value calculation (one-tailed test)

p_value = 1 - stats.norm.cdf(z)

print(f"Observed difference in completion rates: {observed_diff:.4f}")
print(f"Z-score: {z:.4f}")
print(f"P-value: {p_value:.4f}")

# Decision
if p_value < alpha:
    print("Reject the null hypothesis. The new UI design meets the cost-effectiveness threshold.")
else:
    print("Fail to reject the null hypothesis. The new UI design does not meet the cost-effectiveness threshold.")

Observed difference in completion rates: 0.0764
Z-score: 26.1691
P-value: 0.0000
Reject the null hypothesis. The new UI design meets the cost-effectiveness threshold.


#### Conclusion:
P-value is less than significance level, i.e., 0.05 which suggests that it rejects the null hypothesis. Based on it, we can conclude that the new UI design leads to an increase in completion rate that exceeds 5% threshold, making it cost-effective.

In [9]:


# Create figure
fig = go.Figure()

# Bar chart for difference
fig.add_trace(go.Bar(
    x=['Completion Rate Difference'],
    y=[0.0764],
    text=[f'{0.0764:.2%}'],
    textposition='outside',
    marker_color='blue'
))

# Layout configuration
fig.update_layout(
    title='Completion Rate Difference between Test and Control Groups',
    yaxis_title='Difference',
    height=400,
    width=600,
    annotations=[
        dict(
            x=0,
            y=0.0764,
            xref='x',
            yref='y',
            text=f'Z-score: 26.1691<br>p-value: < 0.0001',
            showarrow=True,
            font=dict(size=12),
            align='left',
            arrowhead=2,
            arrowsize=1,
            arrowwidth=2,
            arrowcolor="#636363",
            ax=50,
            ay=-40
        )
    ]
)

fig.show()


### 3. Other Hypothesis

In [8]:
#H0: No difference in the average age of clients engaging with the new process versus the old process
#H1: Difference in the avg. age of clients engaging with the new process versu the old process

In [10]:
# Check for NaN values in relevant columns
print(df[['clnt_age', 'Test', 'Control']].isnull().sum())

print(df['clnt_age'].dtype)


clnt_age    0
Test        0
Control     0
dtype: int64
object


In [11]:
# Ensuring 'clnt_age' is numeric (just in case) and removing any non-finite values
df['clnt_age'] = pd.to_numeric(df['clnt_age'], errors='coerce')

# Separating the results based on the process 
new_process = df[df['Test'] == 1]['clnt_age']
old_process = df[df['Control'] == 1]['clnt_age']

# Check if either group is empty
print(f"New Process Length: {len(new_process)}")
print(f"Old Process Length: {len(old_process)}")

# Only perform t-test if both groups are non-empty
if len(new_process) > 0 and len(old_process) > 0:
    # Performing a 2-sample t-test
    t_stat, p_value = stats.ttest_ind(new_process, old_process, nan_policy='omit')
    
    # Results
    print(f"T-statistic: {t_stat}")
    print(f"P-value: {p_value}")
else:
    print("One of the groups is empty. Cannot perform t-test.")


New Process Length: 177847
Old Process Length: 143462
T-statistic: 7.926400435419626
P-value: 2.263054043650787e-15


### Conclusion:

P-value is less than 0.05 which reject the null hypothesis and conclude that there is significant difference in the average age of clients engaging with new process, i.e., Test group and engaging with old process, i.e., control group

In [13]:
# Creating bar chart
fig = go.Figure(data=[
    go.Bar(
        x=['New Process', 'Old Process'], 
        y=[177847, 143462],
        text=[177847, 143462],
        textposition='auto',
        marker_color=['blue', 'green'],
        width=[0.4, 0.4]  # Reduce bar width
    )
])

# Update layout
fig.update_layout(
    title='Process Length Comparison',
    yaxis_title='Process Length',
    annotations=[
        dict(
            x=0.5,
            y=1.05,
            xref='paper',
            yref='paper',
            text=f'T-statistic: 7.9264<br>P-value: < 0.0001',
            showarrow=False,
            font=dict(size=12)
        )
    ],
    bargap=0.2,  # Reduce gap between bars
    width=600,   # Set overall figure width
    height=400   # Set overall figure height
)

fig.show()
